In [1]:
#CONSTANTS TO RUN THE PAGE

PATH_TO_DATA="/Users/williamcochran/Code/learn_stuff/connect6/data/"
MOVES_TO_DETERMINE_GAME=10
FIRST_MOVE="J10"
BLACK=1
WHITE=-1
EMPTY=0

def swap(color):
    return color*-1



In [2]:
#Let's read some data!
import os

# File finder and reader.. returns a list of moves
def parse_game_file (filename):
    with open(filename, "r") as f:
        lines = f.readlines()
        games = []
        for line in lines:
            if line[0] != "(" and line[-1] != ")":
                print("Found a bad line: {}".format(line))
            trimmed = line[1:-2]
            keys=trimmed.split(';')
            keys = keys[3:]
            moves = [k[2:-1] for k in keys]
            if len(moves)>0:
                games.append(moves)
    return games

# Look for all files in the data path
def read_all_data (path=PATH_TO_DATA):
    games = []
    all_files = os.listdir(path)
    data_files = [file for file in all_files if file.endswith(".data")]
    for file in data_files:
        print ("Reading {}".format(file))
        games.extend(parse_game_file (PATH_TO_DATA + file))
    return games



In [43]:
# Board manipulation helper functions
import random
import copy

def make_board():
    board = []
    for a in range(19):
        b = []
        for c in range(19):
            b.append(EMPTY)
        board.append(b)
    board[9][9] = BLACK
    return board

def fill_with_random(board, stones):
    j = 2
    if len(board) != 19 or any(len(row) != 19 for row in board):
        print(f"Board at index {i} is incorrect before. Length: {len(board)}, Row lengths: {[len(row) for row in board]}")

    while stones > 0:
        x=0
        y=0
        while True:
            x = random.randint(0,18)
            y = random.randint(0,18)
            if board[x][y] == EMPTY:
                break
        
        board[x][y] = BLACK if j < 2 else WHITE
        j = (j+1) % 4
        stones = stones - 1

    if len(board) != 19 or any(len(row) != 19 for row in board):
        print(f"Board at index {i} is incorrect before. Length: {len(board)}, Row lengths: {[len(row) for row in board]}")
        raise KeyboardInterrupt()

    return board

def make_random_board():
    return fill_with_random(make_board(),random.randint(6,100))

def get_stone(move_cluster,spot):
    x = ord(move_cluster[spot]) - ord('a')
    y = ord(move_cluster[spot+1]) - ord('1')
    ans = spot + 2
    if spot+2 < len(move_cluster):
        if move_cluster[spot+2].isdigit():
            ans = ans + 1
            # Be sure to add 1 to the digit since a single digit 
            # number is assumed to start at "1" instead of "0"
            y = (y+1) * 10 + ord(move_cluster[spot+2]) - ord('1')
    return ((x,y),ans)

def count_stones (board):
    return sum([sum(k) for k in board])

def return_moves(move_cluster):
    (stone1,offset) = get_stone(move_cluster,0)
    (stone2,offset) = get_stone(move_cluster,offset)
    return (stone1,stone2)

def pretty_print(board):
    for x in board:
        line = "  "
        for y in x:
            if y == -1:
                line = line + "O "
            elif y == 1:
                line = line + "X "
            else:
                line = line + "+ "
        print(line)

def apply_move(board, stone, value):
    (x,y) = stone
    board[x][y] = value

def make_random_boards(number_to_make):
    answer = []
    while number_to_make > 0:
        if (number_to_make % 10000) == 0:
            print ("Need to make {}".format(number_to_make))
        answer.append(make_random_board())
        number_to_make = number_to_make - 1

    for i,board in enumerate(answer):
        if len(board) != 19 or any(len(row) != 19 for row in board):
            print(f"Board at index {i} is incorrect before. Length: {len(board)}, Row lengths: {[len(row) for row in board]}")
            raise KeyboardInterrupt()
    print (f"Exiting make randos")
    return answer


def build_board_list(game):
    board_list = []
    board = make_board()
    value = WHITE
    for move_cluster in game:
        board = copy.deepcopy(board)
        (stone1,stone2) = return_moves(move_cluster)
        apply_move(board,stone1,value)
        apply_move(board,stone2,value)
        value = swap(value)
        board_list.append(board)

    return board_list

def make_boards_for_list(game_list):
    boards = []
    for game in game_list:
        boards.extend(build_board_list(game))
    return boards

def make_boards_for_wins(games):
    (black_wins,white_wins) = separate_wins(games)
    black_board_list = make_boards_for_list(black_wins)
    white_board_list = make_boards_for_list(white_wins)
    return (black_board_list,white_board_list)

In [ ]:
# time for the machine to do some learning!  This block will try to train a classifier that can
# identify connect 6 boards that arise during game play

import math
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from sklearn.model_selection import train_test_split

games = make_boards_for_list(read_all_data())
print ("Loaded {} games.".format(len(games)))

X = games
y = [1 for _ in range(len(X))]
number_of_randoms = len(X)
print ("Making {} number of randoms".format(number_of_randoms))
random_boards = make_random_boards(number_of_randoms)
X.extend(random_boards)
y.extend([-1 for _ in range(number_of_randoms)])

print("{} {}".format(len(X),len(y)))

# Convert lists to NumPy arrays and reshape X to include a channel dimension
X = np.array(X)  # Your list of 19x19 boards
y = np.array(y)  # Your classifications (1 or -1)

X = np.expand_dims(X, -1)  # Add a single channel dimension
print ("Created datasets")

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print ("Split data")

# Define the CNN model architecture
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(19, 19, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='tanh')  # Output layer with tanh activation
])

print ("Model built")
# Compile the model
model.compile(optimizer='adam',
              loss='mean_squared_error',  # Using MSE as the loss function
              metrics=['accuracy'])

print ("compiling the model ready to train...")

# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")


Reading 11936.data
Reading 166925.data
Reading 15609.data
Reading 35959.data
Reading 36197.data
Reading 142840.data
Reading 14287.data
Loaded 445485 games.
Making 445485 number of randoms
Need to make 440000
Need to make 430000
Need to make 420000
Need to make 410000
Need to make 400000
Need to make 390000
Need to make 380000
Need to make 370000
Need to make 360000
Need to make 350000
Need to make 340000
Need to make 330000
Need to make 320000
Need to make 310000
Need to make 300000
Need to make 290000
Need to make 280000
Need to make 270000
Need to make 260000
Need to make 250000
Need to make 240000
Need to make 230000
Need to make 220000
Need to make 210000
Need to make 200000
Need to make 190000
Need to make 180000
Need to make 170000
Need to make 160000
Need to make 150000
Need to make 140000
Need to make 130000
Need to make 120000
Need to make 110000
Need to make 100000
Need to make 90000
Need to make 80000
Need to make 70000
Need to make 60000
Need to make 50000
Need to make 4000

2024-02-26 10:14:34.066921: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Ultra
2024-02-26 10:14:34.066939: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-02-26 10:14:34.066945: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2024-02-26 10:14:34.067168: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-26 10:14:34.067554: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model built
compiling the model ready to train...
Epoch 1/10
    1/22275 [..............................] - ETA: 2:04:30 - loss: 0.9682 - accuracy: 0.0000e+00

2024-02-26 10:14:34.902478: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


22275/22275 [==============================] - 126s 6ms/step - loss: 0.0042 - accuracy: 0.4995 - val_loss: 0.0011 - val_accuracy: 0.4982
Epoch 2/10
22275/22275 [==============================] - 124s 6ms/step - loss: 0.0032 - accuracy: 0.5000 - val_loss: 0.0137 - val_accuracy: 0.4984
Epoch 3/10
22275/22275 [==============================] - 124s 6ms/step - loss: 0.0089 - accuracy: 0.4992 - val_loss: 8.5174e-04 - val_accuracy: 0.4983
Epoch 4/10
 3865/22275 [====>.........................] - ETA: 1:33 - loss: 0.0015 - accuracy: 0.4963

In [119]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc}")


Epoch 1/10
10852/10852 [==============================] - 68s 6ms/step - loss: 1.7341 - accuracy: 0.3600 - val_loss: 1.9617 - val_accuracy: 0.4566
Epoch 2/10
10852/10852 [==============================] - 68s 6ms/step - loss: 1.6740 - accuracy: 0.3221 - val_loss: 1.6245 - val_accuracy: 0.2555
Epoch 3/10
10852/10852 [==============================] - 67s 6ms/step - loss: 1.7327 - accuracy: 0.1571 - val_loss: 1.8192 - val_accuracy: 0.0576
Epoch 4/10
10852/10852 [==============================] - 68s 6ms/step - loss: 1.8144 - accuracy: 0.0688 - val_loss: 1.7231 - val_accuracy: 0.1538
Epoch 5/10
10852/10852 [==============================] - 68s 6ms/step - loss: 1.7608 - accuracy: 0.1166 - val_loss: 1.8198 - val_accuracy: 0.0748
Epoch 6/10
10852/10852 [==============================] - 69s 6ms/step - loss: 1.7470 - accuracy: 0.1122 - val_loss: 1.8311 - val_accuracy: 0.0544
Epoch 7/10
10852/10852 [==============================] - 69s 6ms/step - loss: 1.7140 - accuracy: 0.1778 - val_loss: 1